# Introduction and rationale

*Prepared by Artem Sotnikov, initial draft 2023-03-18*

For the current design cycle, the descion was made to construct a new parachute. The reasons for this descision were as follows:

1. Legacy knowledge about how parachutes are constructed was not passed one during the COVID era, and some of it was at risk of being lost as the last people who sewed the parachute (Roman, Zach) graduated. Sewing the new parachute would serve as knowledge transfer.
2. The rocket underwent significant modifications and as a result, became much lighter during this design cycle. This meant that the parachute could be made smaller. As this eases packing, volume constraints, tangling risks, and mass, this would be a worthwhile pursuit for the recovery system.
3. The old parachute has begun to deteriorate due to the overwhelmingly large amount of testing conducted on it. If it was chosen to fly this year, it would have possibly needed significant maintenance and cleaning. 

The format of this document is a jupyter notebook in the github simulink workspace due to the necessity of carrying out calculations and also since the calculations may involve the creation of simulink models (although this is admittedly unlikely).

--- 

*Revision 1: 2023-03-21, Artem Sotnikov*

A mistake in the initial draft was caught, which was the lack of a square root term in the nominal diameter scaling equation. After this was corrected, it became clear that the size of the parachute would not descrese significantly, despite a significant reduction in the weight of the rocket. It was decided at this point that the design will be performed to the nominal mass case. No additional safety factors or analysis will be performed on the maximum mass case, with only a check to ensure that it still conform to the DTEG requirements. Some notes on vent sizing have also been summarized from slack threads and ongoing discussion.



In [141]:
# Python initialization

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import math
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy import stats

%load_ext ipydex.displaytools
import ipydex # For better display

%load_ext autoreload
%autoreload 2
%matplotlib widget

from rocketpy.EnvironmentAnalysis import EnvironmentAnalysis
from datetime import datetime

FT_TO_M = 0.3048 ;
LB_TO_KG = 0.453592 ;

The ipydex.displaytools extension is already loaded. To reload it, use:
  %reload_ext ipydex.displaytools
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sizing approach 

The parachute from the previous design cycle was relatively well characterized. It had an unreefed drag area of rouhgly $14 m^2$, against a constructed diameter of 3.8 meters. This value is based on experimental data collected during Truck Test 4 and analyzed during the 2022 competition cycle using the 'truck-test-rapid-analyzer' script, code for which is available on github (https://github.com/waterloo-rocketry/truck-test-rapid-analyzer). The specific analysis file package is also uploaded to the team drive at Recovery/2023/TruckTest4AnalysisRedone. It must be noted that there is uncertainty in this unreefed area due to the necessity of data filtering via hand-picked constants. This is explored to some extent in the podium presentation and can be observed directly by changing the threshold values in the script-generated excel spreadsheet. The $14 m^2$ (corresponding to the 150 ft^2 in the original units of the calculator) is actually a conservative underestimate based on the range of values given by the data analysis, with values going up to 160 - 170 for the higher windspeed cases. Therefore, this lower value can be used with some degree of reliability. This also gives a useful value of the drag coefficient with respect to the constructed-diameter-area (which should stay the same for the new parachute, assuming that their characteristics are the same):

In [142]:
cd_const = 14/(0.25*math.pi*3.8**2) ##:

cd_const := 1.2344427719038975

---

Given that the parachute to be sewn will employ a very similar construction, the best way to size it would be a scaling of the old parachute. In the manual (Parachute Recovery Systems Design Manual, T.W. Knacke, 1992), almost all drag area calculations are ultimately tied to the 'nomial diameter' of the parachute, which is a function of the total fabric surface area. Therefore, the selected approach for sizing the parachute would be to calculate a new nominal diamater assuming the same relative drag coefficient. 

Sizing parachutes could be done liberally to place a large factor of safety on the landing loads. In the author's opinion, such a design philosophy is appropriate for a dual-separation system, which by its nature allows large parachutes and simple parachute bays, where tangling is not a significant concern. It also lends itself well to well-chaterized legacy systems where the risk of main at apogee even is low. However, in a single-bay reefing system, a larger parachute carries a significant penalty to system development and reliability. 

In a newly developed system, a 'main at apogee' is a possible scenario. A larger main would exarcerbate the effects of this scenario and could lead to system loss should the parachute drift into the white sands missile range. It also carries the disadvantage of larger drift in the last 1500 ft, although this is smaller concern. 

The mass that the new parachute would need to target would be 84.2 lbs (which is the expected load as predicted in the cycle 2.2 load analysis). Additionally, a check will need to performed to ensure that the parachute still falls within the allowable envelope even at the maximum load case, which is 92.9 lbs as per the cycle 2.2 load analysis. However, for this check, no additional factors of safety or climactic considerations will be performed, since doing so will necessistate a larger parachute and lead back to the risks of large parachutes as discussed in the previous paragraph 

The laod case is contrasted with the old parachute that was to be used on the projected 100 lb KotS rocket (which ultimately grew to 105.51 lbs), and was sized for a 113 lbs SotS rocket, which was ultimately 104.2 lbs (values taken from the respective reports, available on the drive or the website). 



Due to the nature of the reefing system, only one parachute needs to be sized, with the reefed stage being fully controlable by the length of the reefing line, which is done after the construction of the parachute itself. Thus, the driving factor of the sizing is the ground impact velocity. As per the DTEG, it is recommended at lower than 30 ft/s. Using a 5% margin, ths speed becomes:

In [143]:
target_descent_speed = 30*(1 - 0.05) ##:

target_descent_speed := 28.5

---

As noted in many texts on descent speeds, the descent speed of the parachute is given by:

$V_{desc} = \large \sqrt{\frac{2 m}{C_d S_o \rho}}$

Written in a form that allows for easier corrections due to altitude effects, it is also stated in the manual as:
$\therefore V_{desc} = \large \sqrt{\frac{2 m}{C_d S_o \rho_o}} \cdot \frac{1}{\sqrt{\rho \, / \rho_o}{}}$

Where (as per the original source):

> $m$ is the mass of the vehicle, in lbs  
> $C_d S_o$ is the drag area, in ft^2  
> $\rho$ is the air density, (in slugs/ft^3 !)  
> $\rho_o$ is the air density at sea level (in slugs/ft^3)  

The slug to lbs conversion is given by the acceleration due to gravity, or lbs can be used and the mass must be changed to newtons (i.e. the gravitational force on the system). This makes for easier operations with metric units:

$V_{desc} = \large \sqrt{\frac{2 m g}{C_d S_o \rho}}$

This can also be trivially re-arranged to solve for CdSo:

$C_d S_o = \large\frac{2mg} {\rho V_{desc}^2} $

In [144]:
def v_desc(m, CdSo, rho):
    return math.sqrt(2*m*(9.81)/(CdSo * rho))

def cdso(m, v, rho):
    return 2 * m * (9.81) / (rho * v**2)

Air density exists as a function of altitude and can be found in many tables. Since this is a code-capable document, all desired pressures can be calculated directly using the metric earth atmosphere model provided by NASA: https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html. 

Which states that pressure in (in kPa) can be calculated from temperature (in $^{\circ}$C):

$p = 101.29 \cdot \large (\frac{T + 273.1}{288.08})^{5.256}$  

Where temperature varies directly with altitude as a 'lapse rate', given by $T = 15.04 - 0.00649 \cdot h$
  
And from this, density is $\rho = p \cdot (T + 273.1))^{-1}$

In last year's report, variations in pressure and temperature were considered to model the uncertainty in these values. The methodology used (which was somewhat crude but nontheless valid) revealed that temperature varied by 10 degrees, and pressure itself could vary by about 0.7 kPa due to other weather effects.

This gives an upper and lower bound for pressure at any given altitude (ignoring the recursive variation of temperature on pressure):

$\require{color} \large \rho_{\, upper} =  \small (101.29 \,\large ((\frac{(15.04 - 0.00649 \cdot h) + 273.1}{288.08})^{5.256}) \small \textbf{+ 0.7}) \small \cdot ((15.04 - 0.00649 \cdot h) + 273.1 \textbf{- 10}))^{-1}$ 

$\require{color} \large \rho_{\, lower} = \small (101.29 \,\large ((\frac{(15.04 - 0.00649 \cdot h) + 273.1}{288.08})^{5.256}) \small \textbf{- 0.7}) \small \cdot ((15.04 - 0.00649 \cdot h) + 273.1 \textbf{+ 10}))^{-1}$ 

Another less conservative and more accurate approach is to use the historical values from new mexico directly. This website lists historical averages for June directly:
https://www.timeanddate.com/weather/@5492576/climate. However, it clearly lists them adjusted for altitude, which must be converted back using a conversion factor taken from
https://www.engineeringtoolbox.com/air-altitude-pressure-d_462.html

Then temperature values can be taken from the extremes directly, adding 2.5 degrees to account for additional peak temperature variation over the course of a day. For pressure, the average value can be modulated by adding or subtracting values. A value of 1 ihHg (3.39 Kpa) is given by this source as a very extreme local pressure variation, and 0.5 inHg (1.695) as the average fluctuation.
https://www.infoplease.com/math-science/weather/weather-feeling-the-pressure#:~:text=Generally%2C%20at%20ground%20level%2C%20the,the%20pressure%20will%20be%20lower.

However, this would seem to conflict with the rocketpy environmental modelling of the launch site. Perfoming an analysis of this sort is out of scope for this year, but it provides some interesting numbers in the examples. An even more accurate modelling capability is provided by rocketpy's environmental analysis package, but will not be considered within scope for this analysis. 
https://docs.rocketpy.org/en/latest/notebooks/environment_analysis_class_usage.html

It gives the average daytime surface pressure as 25.16 inHg (85.2 kPa), with a variation of 0.08 inHg (0.27 kPa). This is much less varied than some of the analytical techniques described above. The daily minimum and maximum temperatures roughly align with the historical data, at 35 and 18 degrees celsius respectively. Due to the nature of this dataset being specifically for launch modelling, it will ultimately be used for the historical value - derived air density function.

In [145]:
def calculate_densities_lapse(h):
    T = 15.04 - 0.00649*h

    rho_u = ((101.29*(((T + 273.1)/288.08)**(5.256))) + 0.7) * (0.2869*(T + 273.1 - 10))**(-1)
    rho_avg = ((101.29*(((T + 273.1)/288.08)**(5.256)))) * (0.2869*(T + 273.1))**(-1)
    rho_l = ((101.29*(((T + 273.1)/288.08)**(5.256))) - 0.7) * (0.2869*(T + 273.1 + 10))**(-1)
    return {"rho_u" : rho_u, "rho_avg" : rho_avg , "rho_l" : rho_l}

def calculate_densities_hist():
    
    # avg_p = 101.14 * (1 - 2.25577 * (10**(-5)) * (1401))**5.25588 
    avg_p = 85.2
    p_var = 0.27
    
    rho_u = (avg_p + p_var) * (0.2869*(273.15 + 18 - 2.5))**(-1)
    rho_avg = (avg_p) * (0.2869*(273.15 + 26))**(-1)
    rho_l = (avg_p - p_var) * (0.2869*(273.15 + 35 + 2.5))**(-1)
    
    return {"rho_u" : rho_u, "rho_avg" : rho_avg, "rho_l" : rho_l}


Given the ground elevation at spaceport america, which is 1,401 m, the densities can be calculated:

In [146]:
rho_lapse = calculate_densities_lapse(1401) ##:
rho = calculate_densities_hist() ##:

rho_lapse := {'rho_u': 1.1189855884049338,
 'rho_avg': 1.070141825546099,
 'rho_l': 1.0246776982951848}

---

rho := {'rho_u': 1.0320757976172659,
 'rho_avg': 0.9927046114799415,
 'rho_l': 0.9529260906686793}

---

This reveals that the historical data for the spaceport has a lower average air density than would otherwise be predicted. Due to this, the historical data will be used.

These are the desired densities, since the final descent occurs at the ground. Now executing the calculation for drag areas (in m^2):

In [147]:
target_descent_speed ##: For reference
cdso_l = cdso(84.2*LB_TO_KG, target_descent_speed*FT_TO_M, rho['rho_u']) ##:
cdso_h = cdso(84.2*LB_TO_KG, target_descent_speed*FT_TO_M, rho['rho_l']) ##:

(target_descent_speed) := 28.5

---

cdso_l := 9.621554442669723

---

cdso_h := 10.420717380891716

---

For reference (as stated earlier) the old parachute had a drag area of $14m^2$, and with it the descent speed value (under the low density) would have been:

In [148]:
v_desc(93*LB_TO_KG, 14, rho['rho_l'])/FT_TO_M # in ft/s

25.841342933238806

Sizing the parachute for the lower density case (higher drag area case) can now be done. The last parachute possessed a nominal diamater of 4.84, and this is the value being scaled in the new parachute. Therefore, the new nominal diameter must be:

In [149]:
D_o_new = 4.84 * (cdso_h/14)**(1/2) ##:

D_o_new := 4.175708298154056

---

The nominal diameter is given as the diameter that would be obtained if all of the fabric was laid down in a flat circle. This is a function of the canopy's geometry:

$S = (1/4) \pi D_o^2$  

$D_o = \large\sqrt{\frac{4 \cdot S}{\pi}}$ 


This means that the new surface area must be:

In [150]:
S_new = (1/4)*(math.pi)*D_o_new**2 ##:

S_new := 13.69462632807206

---

For the ellipsoidal shape that is used on the canopy (given by Nakka's website), the minor:major axis ratio is 0.707:1, due to claimed 'structural efficiency' that Nakka doesn't really elaborate on in his website. The major diameter can be referred ot as the 'constructed diameter' or $D_c$. Given the formula for the surface area of the ellipsoid:

![ellipsoid_formula.png](Figures/ellipsoid_formula.png)

In this specific case, (and with only half the ellipsoid) it would be:

$S = 2 \pi ((1/3)(D_c^{3.2} + 2 (0.707 D_c^2)^{1.6}))^{(1/1.6)}$

This is too cumbersome to re-arrange manually, so a numerical solver will be employed:

In [158]:
def SA_ellipse(dc):
    rc = dc/2
    return 2*math.pi*((1/3)*(rc**(3.2) + 2*(0.707*rc**2)**(1.6)))**(1/1.6)


In [156]:
# fsolve works for roots of zero so I just stuck the thing in a lambda function
dc_new = fsolve(lambda dc: SA_ellipse(dc) - S_new, 4.84)[0] ##:

# Sanity check:
SA_ellipse(dc_new) ##:

dc_new := 3.2773909427578323

---

(SA_ellipse(dc_new)) := 13.694626328072113

---

This is the new diameter of the parachute, which is rounded down to 3.25m for simplicity (and admittedly, because that is the size of the gore template we already plotted at the time of writing). With this, Richard Nakka's parachute gore calculator script (shown in a link below) can be used to plot the specific gore pattens:

https://www.nakka-rocketry.net/softw.html#PARA

Aside from the diameter, the only other parameter required by the calculator is the number of gores. For simplicity and increased structural strength, this number will be retained at 16, same as the previous parachute (even though perhaps it could be reduced; but doing an analysis on it at this point is out of scope for this year's sizing due to how delayed it has been). Putting in the parameters into the calculator yields:

![parachute_params_rev2.png](Figures/parachute_params_rev2.png)

Which is the current proposed gore construction for the 2023 parachute. Furthermore, the top of each gore needs to be trimmed in order to produce the vent. Since the approach was to keep the geometry as similar as possible to the previous parachute. Measuring the gores on it, a gore height of about 195 cm was measured. For a 3.8 meter parachute, the full gore template possessed a height of 246 cm. This meant that 51 cm was taken, translating to roughly 20% of the gore. Online reasources such as the following mention the vent size as a function of diamater, so the approach to size it by diameter is followed.  
https://fruitychutes.com/help_for_parachutes/parachute-help/how_to_make_a_parachute.htm#:~:text=Our%20standard%20design%20comes%20with,tape%20to%20ensure%20maximum%20strength   
The same ratio is retained for this parachute, yielding the following trim length and new gore height:


In [153]:
trim_len = 2.10 * 0.20 ##:
new_height = 2.10 * (1 - 0.20) ##:

trim_len := 0.42000000000000004

---

new_height := 1.6800000000000002

---

That locks in all parameters for the 2023 parachute.

As a sanity check, the descent speed with this new constructed diameter and old coefficient of drag is:

In [154]:
v_desc(84.2*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234,  rho['rho_l'])/FT_TO_M ##: feet per second

(v_desc(84.2*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234,  rho['rho_l'])/FT_TO_M) := 28.514312118812466

---

With the maximum mass case, the descent speed is (at both the minumum and average climactic conditions):

In [165]:
v_desc(92.9*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234, rho['rho_avg'])/FT_TO_M ##: feet per second
v_desc(92.9*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234, rho['rho_l'])/FT_TO_M ##: feet per second

(v_desc(92.9*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234, rho['rho_avg'])/FT_TO_M) := 29.345012552297966

---

(v_desc(92.9*LB_TO_KG, (0.25*math.pi*dc_new**2)*1.234, rho['rho_l'])/FT_TO_M) := 29.95123332315653

---

Which is still within bounds of the allowable descent rate